In [1]:
suppressMessages(library("DESeq2"))
suppressMessages(library(BiocParallel))
suppressMessages(library(tidyverse))
register(MulticoreParam(4))

suppressMessages(require(pheatmap))
suppressMessages(require(RColorBrewer))
suppressMessages(require(dendextend))
suppressMessages(require(ComplexHeatmap))
suppressMessages(require(circlize))

SIG_THRESH = 0.01
FC_THRESH = 1.5
WIDTH = 1.6
HEIGHT = 2.6

RES = 300

In [2]:
run_deseq <- function(data_raw, coldata, G1, G2, RUN_TOO = FALSE,CELLTYPES = NULL){
    
    DESIGN = "~groups"
    
    if (RUN_TOO){
        DESIGN = paste0(DESIGN," + ",CELLTYPES)
        dds <- DESeqDataSetFromMatrix(countData=data_raw, 
                                      colData=coldata, 
                                      design= formula(DESIGN)
                                          )
    } else {        
        dds <- DESeqDataSetFromMatrix(countData=data_raw, 
                                      colData=coldata, 
                                      design=formula(DESIGN)
                                     )
    }

    dds <- DESeq(dds,parallel=TRUE)

    res <- results(dds, contrast = c("groups",G1,G2))

    tmp <- res %>% data.frame() %>% filter(abs(log2FoldChange)>1.5) %>% filter(padj < 0.01)
    cat(nrow(tmp))
    
    return(res)
}

In [3]:
create_heatmap <- function(res,groups,cpm,meta_data) {
    
    ANNOTATIONS = c("Diagnosis","severity",'group')
#     ANNOTATIONS = c("Row.names","groups")
    
    ###----------------------------------
    ## Extract significant genes
    sig_genes <- data.frame(res) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% rownames(gene_id)
#             rownames_to_column(var = "gene_id_ensmbl",) %>% mutate(gene_id = gsub(".*_","",gene_id_ensmbl)) %>% pull(gene_id)

    ###----------------------------------
    ## Subset metadata 
    metadata <- meta_data %>% 
        column_to_rownames(var = "Row.names") %>%
        mutate(severity = ifelse(grepl("ontrol",severity),NA,severity)) %>%
        rename(Diagnosis = groups) %>%
        select(all_of(ANNOTATIONS))
    
    colnames(metadata) <- c("Diagnosis","Severity","group")
    
    ###----------------------------------
    ## Subset count matrix
    mat <- data.frame(cpm) %>% 
            filter(row.names(cpm) %in% all_of(sig_genes)) %>% 
            select(all_of(rownames(metadata))) %>% 
            as.matrix()

    mat <- t(scale(t(mat)))

    ###----------------------------------
    ## Colors
    color = colorRampPalette(c("blue","yellow"))(50)
    breaksList = seq(-2, +2, length = 51)

    #color_groups = c('COVID-19\ndiscovery' = '#c1272d', 'MIS-C\ndiscovery' = '#0000a7', 'Control_Non-inflammatory\ndiscovery' = '#eecc16',"MIS-C\nvalidation" = '#008176')
    
    my_colour <- list(
    Diagnosis = c('COV' = '#F0484E', 'M' = '#5CB2EB', 'C' = '#FBE77C') , #"MISC_acute_validation" = '#00FFFF'),
    Severity = c("-1" = "white", "0" = "white", "2" = "#efe5d7", "3" = "#bc8e52"),
    group = c("validation" = "orange", "discovery" = "light blue", "UCSF" = "forest green") #,
    # PCR = c("1" = "green", "2" = "blue"),
    # Antibody =c("1" = "green", "2" = "blue")
    )

    ###----------------------------------
    ## Plot
    
#     return(list(mat, metadata))
    
    # pdf("./tmp.pdf")
    heatmap_plt <- pheatmap(mat,

             # Colors
             col=color,
             breaks=breaksList, 
             annotation_col=metadata,
             annotation_colors=my_colour,
             na_col = "#FFFFFF",

             # Fonts
             show_colnames=F,
             show_rownames=F,
             fontsize=12,
             fontsize_col=3,
             annotation_names_col=F,
             annotation_names_row=F,

             # Clustering

             clustering_distance_cols="correlation",
             clustering_distance_rows="correlation", 
    #          clustering_distance_cols="euclidean",
    #          clustering_distance_rows="euclidean",
    #          cluster_cols=hc,
    #          cluster_rows=hr,
    #          clustering_distance_rows="euclidean",
    #          clustering_method="complete",
             treeheight_row=0,
            treeheight_col= 15,

             # Misc.
             border_color=NA,
            legend=FALSE,
            annotation_legend=FALSE
            ) 
    # dev.off()




    return(heatmap_plt)

        }


CELLTYPE_labs = "wbc"

In [25]:
### combine (dont need mdf3)
labdata <- read.delim("../1_sample-data/paired_sample_key.tsv") %>% select(wbrna_sample_id,wbc,crp,alt,alb)


################################
## read in wbrna key
wbrna_meta <- read.csv("../1_sample-data/STable7_wbrna-samples.csv")


# wbrna_key %>% head()


################################
## read in wbrna key
labdata  <- merge(labdata, wbrna_meta, by="wbrna_sample_id", all.y= TRUE) #%>%
#         filter(!is.na(wbc)) %>%
#         filter(wbrna_id %in% colnames(data_raw)) #%>% 
#         arrange(siteid,wbrna_date_collection) %>% #%>%select(wbrna_id,wbc)

labdata <- labdata[!duplicated(labdata$PTID),] 

labdata %>% head()

,wbrna_sample_id,wbc,crp,alt,alb,PTID,group,Diagnosis,severity,timepoint,⋯,Intestinal.secretory.cell,Intestinal.tuft.cell,Type.II.Pneumocyte,Cell.of.skeletal.muscle,Schwann.cell,Tendon.cell,Mesothelial.cell,Plasmablast,Kidney.epithelial.cell,Thymocyte
,<chr>,<dbl>,<dbl>,<int>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,PV1,NA,NA,NA,NA,Patient218,discovery,COVID-19,2,acute,⋯,1.33e-05,0.01898214,1.26e-05,0.000048100,1.78e-05,1.16e-05,0.000019500,0.000004470,8.16e-06,0.003373379
2,PV100,5.14,16.2,41,2.4,Patient77,discovery,MIS-C,3,acute,⋯,1.94e-06,0.00004980,1.29e-06,0.000443479,2.48e-06,1.27e-06,0.000133765,0.000003300,1.30e-06,0.000002170
3,PV101,11.53,12.7,31,3.3,Patient79,discovery,MIS-C,2,acute,⋯,1.58e-05,0.03062372,6.51e-05,0.014795700,1.93e-05,6.61e-06,0.000001880,0.000007830,1.36e-05,0.064434426
4,PV102,9.10,156.2,43,2.3,Patient80,discovery,MIS-C,3,acute,⋯,9.92e-06,0.04879090,1.61e-05,0.013138938,9.93e-05,1.43e-05,0.000002250,0.005421059,5.08e-05,0.045913098
5,PV103,9.36,10.2,27,3.9,Patient82,discovery,MIS-C,2,acute,⋯,7.48e-06,0.02249626,1.08e-05,0.001680164,1.36e-05,1.03e-05,0.000002380,0.004566034,6.23e-06,0.009268024
6,PV104,9.95,8.2,24,1.9,Patient83,discovery,MIS-C,3,acute,⋯,1.56e-06,0.00931138,4.54e-06,0.008044692,7.72e-05,2.72e-06,0.000051500,0.063632476,2.30e-06,0.000003370


In [31]:
### read in data
mdf_raw = read.csv("../../revisions/PreVAILCDEs-PlasmaProteinMarkers_DATA_2022-10-17_0827.csv")

### manual fixing
mdf_raw <- mdf_raw %>% 
    filter(!(study_id %in% c(160,161) & redcap_repeat_instance ==2)) %>%
    mutate(siteid = toupper(siteid))


### Split and combine
not_all_na <- function(x) any(!is.na(x))

mdf1 <- mdf_raw %>% filter(redcap_repeat_instrument == "") %>% 
                    select(!c(redcap_repeat_instrument, redcap_repeat_instance))%>%
                    select(study_id,siteid)
#                     select_if(~ !any(is.na(.)))  #%>% colnames()

mdf2 <- mdf_raw %>% filter(redcap_repeat_instrument == "labs_and_microbiology") %>% 
                    select(!c(redcap_repeat_instrument, redcap_repeat_instance))%>%
                    select(!siteid)
#                     select_if(~ !any(is.na(.)))  #%>% colnames()


### combine (dont need mdf3)
labdata <- merge(mdf1, mdf2, by = "study_id") %>% rename(ldh_date = ldhdate,
                     ferritin_date = ferr_date,
                     alb_date = albumin_date) %>%
            select(siteid, wbc, wbc_date)

# labdata %>% head()


################################
## read in wbrna key
wbrna_key <- read.csv("../1_sample-data/STable7_wbrna-samples.csv") 

# wbrna_key2 <- read.csv("../../marker_genes/meta_data/data/UCSF_PREVAIL_METADATA_EXP4.csv") %>%
#                 rename(wbrna_id = PV.,
#                        wbrna_date_collection = Date,
#                        PTID = Patient.ID) %>%
#                 mutate(PTID = toupper(PTID),
#                       wbrna_date_collection = as.Date(wbrna_date_collection, format = "%m/%d/%y")) %>%
#                 select(PTID,wbrna_date_collection,wbrna_id)

# wbrna_key <- rbind(wbrna_key, wbrna_key2) 

wbrna_key %>% head()


################################
## read in wbrna key
# labdata  <- merge(labdata, wbrna_key, by = "wbrna_sample_id", all.y= TRUE) #%>%
#         filter(!is.na(wbc)) %>%
#         filter(wbrna_id %in% colnames(data_raw)) #%>% 
#         arrange(siteid,wbrna_date_collection) %>% #%>%select(wbrna_id,wbc)

# labdata <- labdata[!duplicated(labdata$siteid),] 
labdata %>% head()



,PTID,wbrna_sample_id,group,Diagnosis,severity,timepoint,reads_sequenced,transcriptome_cov,ivig,days_ivig_before_samp,⋯,Intestinal.secretory.cell,Intestinal.tuft.cell,Type.II.Pneumocyte,Cell.of.skeletal.muscle,Schwann.cell,Tendon.cell,Mesothelial.cell,Plasmablast,Kidney.epithelial.cell,Thymocyte
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Patient3,PV351,validation,MIS-C,3,acute,38098558,25.44,1,-2,⋯,8.35e-06,0.10954676,1.98e-05,0.015621141,1.06e-05,1.10e-05,6.70e-06,0.000011500,1.53e-05,0.021315977
2,Patient6,PV352,validation,MIS-C,3,acute,26489040,20.56,1,NA,⋯,1.25e-05,0.08352631,1.78e-05,0.014168603,1.86e-05,1.51e-05,1.27e-05,0.000017600,1.15e-05,0.000160293
3,Patient7,PV353,validation,MIS-C,3,acute,26711938,26.72,1,-3,⋯,4.86e-06,0.08213655,5.90e-06,0.023575040,4.38e-06,4.18e-06,4.32e-06,0.000005290,3.43e-06,0.000024300
4,Patient16,PV358,validation,MIS-C,3,acute,26069978,21.42,1,-3,⋯,1.82e-05,0.04287833,8.03e-05,0.011497580,1.53e-05,2.02e-05,9.12e-06,0.000048900,1.46e-05,0.000080100
5,Patient21,PV359,validation,MIS-C,3,acute,28885786,28.56,1,-3,⋯,6.31e-06,0.09026610,7.14e-06,0.022972576,7.65e-06,1.93e-05,2.21e-06,0.000407147,4.43e-06,0.004639198
6,Patient22,PV364,validation,MIS-C,3,acute,33887900,26.18,1,-2,⋯,2.91e-06,0.09142923,4.51e-06,0.002670228,3.29e-06,4.82e-06,2.08e-06,0.002681353,2.62e-06,0.000046400


,siteid,wbc,wbc_date
,<chr>,<dbl>,<chr>
1,UCSF0114,28.7,2021-09-11
2,UCSF0039,8.7,
3,UCSF0015,5.0,
4,UCSF0066,8.8,
5,UCSF0153,6.5,
6,UCSF0156,5.1,


---
## COVID19 vs Controls

In [27]:
##-------------------------
# Read in data
data_raw= read.table("..//1_sample-data/2covid_ctrls_counts.txt", header = TRUE, row.names=1,check.names=FALSE)

##-------------------------
# Make coldata
groups <- factor(c(rep("COV",37), rep("C",26)))

coldata <- data.frame(row.names=colnames(data_raw), groups)
# coldata <- merge(coldata,decon, by.x = 0, by.y = "SEQ_ID..UCSFonly.")

coldata <- merge(coldata,labdata[,c("wbrna_sample_id","wbc")], by.x = 0, by.y = "wbrna_sample_id", all.x = TRUE)
coldata[is.na(coldata$wbc),"wbc"] <- 7500

rownames(coldata) <- coldata$Row.names
coldata <- coldata[colnames(data_raw),]


##-------------------------
# Run DESeq2
cat("DEGs w/o CTO: ")
res <- suppressMessages(run_deseq(data_raw, coldata, "COV","C", FALSE))

cat("\n")

cat("DEGs w/ CTO: ")
res_cto <- suppressMessages(run_deseq(data_raw, coldata,"COV","C", TRUE, CELLTYPE_labs))

DEGs w/o CTO: 699
DEGs w/ CTO: 104

In [23]:
prefix = "covid_controls_labs"

write.table(res, paste0("./dea_outputs/",prefix,".txt"), sep = "\t", quote=FALSE)
write.table(res_cto, paste0("./dea_outputs/",prefix,"_cto.txt"), sep = "\t", quote=FALSE)

---
## MISC vs Controls

In [25]:
##-------------------------
# Read in data
data_raw = read.table("../../1_sample-output/2misc_ctrls_counts.txt", header = TRUE, row.names=1,check.names=FALSE)

##-------------------------
# Make coldata
groups <- factor(c(rep("M",72 ), rep("C",26)))

coldata <- data.frame(row.names=colnames(data_raw), groups)

coldata <- merge(coldata,labdata[,c("wbrna_sample_id","wbc")], by.x = 0, by.y = "wbrna_sample_id", all.x = TRUE)
coldata[is.na(coldata$wbc),"wbc"] <- 7500

rownames(coldata) <- coldata$Row.names
coldata <- coldata[colnames(data_raw),]

##-------------------------
# Run DESeq2
cat("DEGs w/o CTO: ")
res2 <- suppressMessages(run_deseq(data_raw, coldata, "M","C", FALSE))

cat("\n")

cat("DEGs w/ CTO: ")
res_cto2 <- suppressMessages(run_deseq(data_raw, coldata,"M","C", TRUE, CELLTYPE_labs))

DEGs w/o CTO: 1051
DEGs w/ CTO: 889

In [26]:
prefix = "misc_controls_labs"

write.table(res2, paste0("./dea_outputs/",prefix,".txt"), sep = "\t", quote=FALSE)
write.table(res_cto2, paste0("./dea_outputs/",prefix,"_cto.txt"), sep = "\t", quote=FALSE)

---
## MISC vs COVID19

In [28]:
##-------------------------
# Read in data
data_raw = read.table("../../1_sample-output/2misc_covid_counts.txt", header = TRUE, row.names=1,check.names=FALSE)

##-------------------------
# Make coldata
groups <- factor(c(rep("M",72 ), rep("COV",37)))

coldata <- data.frame(row.names=colnames(data_raw), groups)
# coldata <- merge(coldata,decon, by.x = 0, by.y = "SEQ_ID..UCSFonly.")

coldata <- merge(coldata,labdata[,c("wbrna_sample_id","wbc")], by.x = 0, by.y = "wbrna_sample_id", all.x = TRUE)
coldata[is.na(coldata$wbc),"wbc"] <- 7500

rownames(coldata) <- coldata$Row.names
coldata <- coldata[colnames(data_raw),]

##-------------------------
# Run DESeq2

cat("DEGs w/o CTO: ")
res3 <- suppressMessages(run_deseq(data_raw, coldata, "M","COV", FALSE))

cat("\n")

cat("DEGs w/ CTO: ")
res_cto3 <- suppressMessages(run_deseq(data_raw, coldata,"M","COV", TRUE, CELLTYPE_labs))

DEGs w/o CTO: 75
DEGs w/ CTO: 70

In [29]:
prefix = "misc_covid_labs"

write.table(res3, paste0("./dea_outputs/",prefix,".txt"), sep = "\t", quote=FALSE)
write.table(res_cto3, paste0("./dea_outputs/",prefix,"_cto.txt"), sep = "\t", quote=FALSE)



---
## Pheatmaps

In [34]:
covid_control_labs <- read.delim("./dea_outputs/covid_controls_labs_cto.txt")
misc_control_labs <- read.delim("./dea_outputs/misc_controls_labs_cto.txt")
misc_covid_labs <- read.delim("./dea_outputs/misc_covid_labs_cto.txt")

### Read in metadata
meta_data_all <- read.csv("../1_sample-data/STable7_wbrna-samples.csv")

In [40]:
misc_control_labs %>% head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000419.14_DPM1,71.876681,0.4701054,0.1287523,3.651238,2.609789e-04,1.514982e-03
ENSG00000000457.14_SCYL3,41.084756,-1.1533634,0.1944868,-5.930290,3.023994e-09,9.165429e-08
ENSG00000000460.17_C1orf112,26.077841,-0.5507325,0.2248101,-2.449768,1.429483e-02,4.233489e-02
ENSG00000000938.13_FGR,1225.451619,0.9053390,0.1732586,5.225362,1.738148e-07,2.926129e-06
ENSG00000000971.16_CFH,4.483282,-1.1707960,0.4665776,-2.509328,1.209613e-02,3.713257e-02
ENSG00000001036.14_FUCA2,32.997927,-0.3599068,0.1667064,-2.158927,3.085585e-02,7.947232e-02


In [41]:
create_heatmap <- function(res,groups,cpm,meta_data) {
    
    ANNOTATIONS = c("Diagnosis","severity",'group','ivig_rel_samp')

    
    ###----------------------------------
    ## Extract significant genes
    sig_genes <- data.frame(res) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% rownames()

    ###----------------------------------
    ## Subset metadata 
    metadata <- meta_data %>% 
        filter((Diagnosis %in% all_of(groups)) & (timepoint %in% c("Not-hospitalized","acute"))) %>% 
        column_to_rownames(var = "wbrna_sample_id") %>%  
        mutate(severity = ifelse(grepl("ontrol",severity),NA,severity)) %>%
        select(all_of(ANNOTATIONS))

    colnames(metadata) <- c("Diagnosis","Severity","group","IVIG")

    ###----------------------------------
    ## Subset count matrix
    mat <- data.frame(cpm) %>% 
            filter(row.names(cpm) %in% all_of(sig_genes)) %>% 
            select(all_of(rownames(metadata))) %>% 
            as.matrix()

    mat <- t(scale(t(mat)))

    ###----------------------------------
    ## Colors
    color = colorRampPalette(c("blue","yellow"))(50)
    breaksList = seq(-2, +2, length = 51)

    #color_groups = c('COVID-19\ndiscovery' = '#c1272d', 'MIS-C\ndiscovery' = '#0000a7', 'Control_Non-inflammatory\ndiscovery' = '#eecc16',"MIS-C\nvalidation" = '#008176')
    
    my_colour <- list(
    Diagnosis = c('COVID-19' = '#F0484E', 'MIS-C' = '#5CB2EB', 'Control_Non-inflammatory' = '#FBE77C'), #"MISC_acute_validation" = '#00FFFF'),
    Severity = c("-1" = "white", "0" = "white", "2" = "#efe5d7", "3" = "#bc8e52"),
    group = c("validation" = "orange", "discovery" = "light blue", "UCSF" = "light blue"),
    IVIG = c("after" = "maroon", "before" = "darkseagreen2", "concurrent with" = "dark blue","noivig"="white")#, "forest green"
    # PCR = c("1" = "green", "2" = "blue"),
    # Antibody =c("1" = "green", "2" = "blue")
    )

    ###----------------------------------
    ## Plot

    # pdf("./tmp.pdf")
    heatmap_plt <- pheatmap(mat,

             # Colors
             col=color,
             breaks=breaksList, 
             annotation_col=metadata,
             annotation_colors=my_colour,
             na_col = "#FFFFFF",

             # Fonts
             show_colnames=F,
             show_rownames=F,
             fontsize=12,
             fontsize_col=3,
             annotation_names_col=F,
             annotation_names_row=F,

             # Clustering

             clustering_distance_cols="correlation",
             clustering_distance_rows="correlation", 
    #          clustering_distance_cols="euclidean",
    #          clustering_distance_rows="euclidean",
    #          cluster_cols=hc,
    #          cluster_rows=hr,
    #          clustering_distance_rows="euclidean",
    #          clustering_method="complete",
             treeheight_row=0,
            treeheight_col= 15,

             # Misc.
             border_color=NA,
            legend=FALSE,
            annotation_legend=FALSE
            ) 
    # dev.off()




    return(heatmap_plt)

        }

In [37]:
data_raw1 = read.table("../1_sample-data/2covid_ctrls_counts.txt", header = TRUE, row.names=1,check.names=FALSE)


### Read ftcount matrix
raw_ftcount <- read.delim("../1_sample-data/wbrna_ftcounts.txt",row.names = 1)
nrow(raw_ftcount)

raw_ftcount <- raw_ftcount[rownames(data_raw1),]

### Read gene list to remove
gene.list <- read.delim("../0_support-files/genelist.remove.tsv",col.names = c("type,","ENSMBL","gene_symbol"))
gene.ids <- gsub("\\..*","",rownames(raw_ftcount))

### Subset count matrix
exclude.idx <- gene.ids %in% gene.list[,2]
raw_ftcount = raw_ftcount[!exclude.idx,]          

nrow(raw_ftcount)

raw_ftcount <- raw_ftcount[,colSums(raw_ftcount) > 0]

cpm_ftcount <- edgeR::cpm(raw_ftcount)
head(cpm_ftcount)

[1] 60708

[1] 19026

,PV156,PV157,PV158,PV159,PV160,PV162,PV163,PV164,PV166,PV167,⋯,PV90,PV91,PV92,PV93,PV94,PV95,PV96,PV97,PV98,PV99
ENSG00000000419.14_DPM1,139.53659,14.04847,18.680008,24.090639,35.094615,26.055207,34.22971,19.40313,16.207618,26.2747933,⋯,45.47797,50.562705,37.357891,64.81301,52.071769,46.323305,28.211549,39.3073364,31.462680,40.762215
ENSG00000000457.14_SCYL3,146.88062,37.82279,45.143353,14.454383,52.641923,34.740276,36.86277,16.02867,20.066575,21.9752816,⋯,0.00000,18.669306,8.965894,21.60434,8.842376,21.714049,18.807699,16.2356389,31.462680,7.764231
ENSG00000000460.17_C1orf112,0.00000,12.96781,20.236676,12.045319,21.056769,23.739189,50.02804,16.02867,18.522992,18.1534935,⋯,45.47797,6.223102,4.731999,0.00000,10.807348,5.790413,10.747257,8.5450731,2.458022,38.821157
ENSG00000000938.13_FGR,198.28884,248.54977,236.613439,521.562326,268.724481,236.812881,139.55191,343.35097,334.957440,470.0799375,⋯,1394.65785,727.714002,529.983941,432.08676,613.071389,256.708314,377.497394,493.9052266,904.552060,277.571271
ENSG00000000971.16_CFH,0.00000,0.00000,9.340004,0.000000,1.002703,4.053032,0.00000,0.00000,5.402539,0.4777235,⋯,0.00000,1.944719,1.245263,0.00000,0.000000,3.860275,5.373628,0.8545073,2.458022,1.941058
ENSG00000001036.14_FUCA2,14.68806,35.66149,24.906678,3.613596,14.037846,17.949143,52.66110,10.96698,21.610157,22.4530051,⋯,0.00000,19.058250,19.426103,21.60434,15.719779,10.615757,17.464292,18.7991609,28.021450,17.469521


In [38]:
SIG_THRESH = 0.01
FC_THRESH = 1.5

# SIG_THRESH = 0.05
# FC_THRESH = 0

ANNOTATIONS <- c("Diagnosis","severity")

WIDTH = 1.6
HEIGHT = 2.6

RES = 300

## Make the heatmaps

In [42]:
# covid_control_plt <- create_heatmap(covid_control,c(2,4, 31, 34, 36),cpm_ftcount,meta_data_all)
# misc_control_plt <- create_heatmap(misc_control,c(1,3, 31, 57,58, 33, 35),cpm_ftcount,meta_data_all)
# misc_covid_plt <- create_heatmap(misc_covid,c(1,3, 2,4, 57, 58, 33, 35, 34, 36),cpm_ftcount,meta_data_all)

# covid_control_cto_plt <- create_heatmap(covid_control_cto,c(2,4, 31, 34, 36),cpm_ftcount,meta_data_all)
# misc_control_cto_plt <- create_heatmap(misc_control_cto,c(1,3, 31, 57,58, 33, 35),cpm_ftcount,meta_data_all)
# misc_covid_cto_plt <- create_heatmap(misc_covid_cto,c(1,3, 2,4, 57, 58, 33, 35, 34, 36),cpm_ftcount,meta_data_all)


covid_control_labs_plt <- create_heatmap(covid_control_labs,c("COVID-19","Control_Non-inflammatory"),cpm_ftcount,meta_data_all)
misc_control_labs_plt <- create_heatmap(misc_control_labs,c("MIS-C","Control_Non-inflammatory"),cpm_ftcount,meta_data_all)
misc_covid_labs_plt <- create_heatmap(misc_covid_labs,c("MIS-C","COVID-19"),cpm_ftcount,meta_data_all)

In [43]:
##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "covid-control"
       
png(file=paste0("plots/SupPanelE_",prefix,".png"),
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(covid_control_labs_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "misc-control"

png(file=paste0("plots/SupPanelE_",prefix,".png"),
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(misc_control_labs_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "misc-covid"

png(file=paste0("plots/SupPanelE_",prefix,".png"),
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(misc_covid_labs_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

png 
  2

png 
  2

png 
  2